# Análisis de reingresos hospitalarios para pacientes con diabetes. 

Etapa de modelamiento y hallazgos

Se decidió  ajustar modelos de clasificación con el ibjetivo de encontrar las variables que más contribuyan a la prediccón correcta de los reingresos en menos de 30 días.

In [2]:
# librerias requeridas:

import os
import pandas as pd
import numpy as np
import plotly.express as px
from datetime import datetime

In [4]:
# Lectura de los datos originales:
print(datetime.now())

data=pd.read_csv('G:\\Shared drives\\Books_and_Code_for_Colab_(R and Python)\\ifood_technical_test\\ifood_diabetes_project\\Data\\diabetic_data.csv',na_values="?")
data.shape
data.head()

2020-12-28 13:58:33.860909


C:\Users\emilio\.conda\envs\TMO_3_7\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),NaN,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),NaN,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),NaN,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),NaN,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),NaN,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


In [86]:
print(datetime.now())
# data.info()

2020-12-30 03:32:50.573475


In [7]:
print(datetime.now())
len(data.patient_nbr.unique())

2020-12-28 14:08:05.380842


71518

In [8]:
data.A1Cresult.value_counts()

None    84748
>8       8216
Norm     4990
>7       3812
Name: A1Cresult, dtype: int64

## Lectura de datos "pre-procesados"

Este archivo proviene del proceso de limpieza y exploración desarrollado en `R`

In [15]:
# Lectura de los datos limpios:
print(datetime.now())

data_cleaned=pd.read_csv('G:\\Shared drives\\Books_and_Code_for_Colab_(R and Python)\\ifood_technical_test\\ifood_diabetes_project\\Data\\diabetic_data_cleaned.csv',na_values="?")
data_cleaned.shape
data_cleaned.head()

2020-12-30 01:31:06.998863


,Medical_speciality,Age_,race_,Admission,Discharge,Reaction,Pri_diag,Readmit,time_in_hospital
0,Cardiology,"[30, 60)",Caucasian,Other,Home,None,Circulatory,1,8
1,Surgery,"[30, 60)",Caucasian,Other,Home,None,Musculoskeletal,0,2
2,InternalMedicine,"[60, 100)",Caucasian,Emergency,Other,Norm,Injury,0,4
3,InternalMedicine,"[60, 100)",Caucasian,Emergency,Home,High&Ch,Others,0,3
4,InternalMedicine,"[30, 60)",AfricanAmerican,Emergency,Home,None,Genitourinary,0,5


In [16]:
data_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69990 entries, 0 to 69989
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Medical_speciality  69990 non-null  object
 1   Age_                69990 non-null  object
 2   race_               69990 non-null  object
 3   Admission           69990 non-null  object
 4   Discharge           69990 non-null  object
 5   Reaction            69990 non-null  object
 6   Pri_diag            69990 non-null  object
 7   Readmit             69990 non-null  int64 
 8   time_in_hospital    69990 non-null  int64 
dtypes: int64(2), object(7)
memory usage: 4.8+ MB


A continuación se entandariza la variable numérica `time_in_hospital` y se recodifican las demás variables caregóricas.

In [28]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
sc.fit(data_cleaned["time_in_hospital"].values.reshape(-1, 1))
time_in_hospital_std = sc.transform(data_cleaned["time_in_hospital"].values.reshape(-1, 1))

In [63]:
data_cleaned.columns


Index(['Medical_speciality', 'Age_', 'race_', 'Admission', 'Discharge',
       'Reaction', 'Pri_diag', 'Readmit', 'time_in_hospital'],
      dtype='object')

In [74]:
X_imb = pd.concat([pd.DataFrame(time_in_hospital_std, columns=["time_in_hospital_std"]),     
                     pd.get_dummies(data_cleaned['Medical_speciality'], prefix='Medical_speciality', drop_first=True),\
                     pd.get_dummies(data_cleaned['Age_'], prefix='Age', drop_first=True),\
                     pd.get_dummies(data_cleaned['race_'], prefix='race', drop_first=True),\
                     pd.get_dummies(data_cleaned['Admission'], prefix='Admission', drop_first=True),\
                     pd.get_dummies(data_cleaned['Discharge'], prefix='Discharge', drop_first=True),\
                     pd.get_dummies(data_cleaned['Reaction'], prefix='Reaction', drop_first=True),\
                     pd.get_dummies(data_cleaned['Pri_diag'], prefix='Pri_diag', drop_first=True),],axis=1)

In [75]:
y_imb=data_cleaned['Readmit']

Como se muestra a continuación, la **clase positiva** está altamente desbalanceada ($\approx\ 9\%$), por lo cual el desempeño de cualquier clasificador será enngañoso. Para corregir este problema se opta por aumentar el número de casos con un muestreo aleatorio con repetición.

In [79]:
y_imb.value_counts()#/y_imb.shape[0]


0    63705
1     6285
Name: Readmit, dtype: int64

In [80]:
from sklearn.utils import resample

print('Number of class 1 samples before:', X_imb[y_imb == 1].shape[0])

X_upsampled, y_upsampled = resample(X_imb[y_imb == 1],
                                    y_imb[y_imb == 1],
                                    replace=True,
                                    n_samples=X_imb[y_imb == 0].shape[0],
                                    random_state=123)

print('Number of class 1 samples after:', X_upsampled.shape[0])

Number of class 1 samples before: 6285
Number of class 1 samples after: 63705


Se puede observar como se equilibró el número de registros para cada una de las dos clases

In [85]:
X_bal = np.vstack((X_imb[y_imb == 0], X_upsampled))
y_bal = np.hstack((y_imb[y_imb == 0], y_upsampled))
print(X_bal.shape)
print(y_bal.shape)

(127410, 25)
(127410,)


## Ajuste de modelos

Se pretende construir y comparar los siguientes modelos de clasificación binaria:

* Regresión Logística 
* Arbol de decisión

Para poder comparar el desempeño de los modelos se utiliza la estrategia de **validación cruzada anidada**

In [87]:
## Separación de los conjunto de datos para entrenamiento y prueba:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = \
    train_test_split(X_bal, y_bal, 
                     test_size=0.20,
                     stratify=y_bal,
                     random_state=1)

### Modelo de regresión logística

Se inicia con una prueba donde aun no se usa la validación cruzada.

In [98]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

pipe_lr = make_pipeline(LogisticRegression(random_state=12345, max_iter=500))

pipe_lr.fit(X_train, y_train)
y_pred = pipe_lr.predict(X_test)
print('Train Accuracy: %.3f' % pipe_lr.score(X_train, y_train))
print('Test Accuracy: %.3f' % pipe_lr.score(X_test, y_test))

Train Accuracy: 0.579
Test Accuracy: 0.581


Se observa un desempeño muy pobre. Para evitar la ocurrencia de un subajuste, se trabaja con una validación cruzada que internamente trata de optimizar el parámetro de regularización inversa $C$.

In [129]:
%%time
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
gs = GridSearchCV(estimator=pipe_lr,
                  param_grid=[{'logisticregression__C': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]}],
                  scoring='f1',
                  cv=5)

scores = cross_val_score(gs, X_train, y_train, 
                         scoring='roc_auc', cv=10)
print('CV roc_auc: %.3f +/- %.3f' % (np.mean(scores), 
                                      np.std(scores)))

scores = cross_val_score(gs, X_train, y_train, 
                         scoring='accuracy', cv=10)
print('CV accuracy: %.3f +/- %.3f' % (np.mean(scores),  np.std(scores)))

CV roc_auc: 0.606 +/- 0.007
CV accuracy: 0.579 +/- 0.005
Wall time: 6min 26s


Los resultados muestran que el desempeño no mejoró y que tal vez es necesario considerar interacciones las variables (lo cual no se hará por el momento)

### Arboles de decisión.

In [128]:
%%time
from sklearn.tree import DecisionTreeClassifier

gs = GridSearchCV(estimator=DecisionTreeClassifier(random_state=0),
                  param_grid=[{'max_depth': [1, 2, 3, 4, 5, 6, 7,10,15,20, None]}],
                  scoring='f1',
                  cv=5)

scores = cross_val_score(gs, X_train, y_train, 
                         scoring='roc_auc', cv=10)
print('CV roc_auc: %.3f +/- %.3f' % (np.mean(scores),  np.std(scores)))

scores = cross_val_score(gs, X_train, y_train, 
                         scoring='accuracy', cv=10)
print('CV accuracy: %.3f +/- %.3f' % (np.mean(scores),  np.std(scores)))

# print(gs.best_params_)

CV roc_auc: 0.810 +/- 0.003
CV accuracy: 0.724 +/- 0.004
Wall time: 3min 43s


El modelo de árbol de decisión mejoró el desempeño en la clasificación de manera apreciable 

In [132]:
gs = GridSearchCV(estimator=DecisionTreeClassifier(random_state=0),
                  param_grid=[{'max_depth': [1, 2, 3, 4, 5, 6, 7,10,15,20, None]}],
                  scoring='f1',
                  cv=5)
gs = gs.fit(X_train, y_train)
print(gs.best_score_)
print(gs.best_params_)

0.7371216051723815
{'max_depth': None}


Ahora se ajustará el árbol con `'max_depth'= None`, usando todos los datos del conjunto de entrenamiento.

In [141]:
# !pip install pydotplus
from io import StringIO
from sklearn.tree import export_graphviz
import pydotplus
from IPython.display import Image, SVG

clf = DecisionTreeClassifier(max_depth = None)

# Train Decision Tree Classifer

clf = clf.fit(X_train, y_train)

In [147]:
feat_labels = X_imb.columns

In [148]:
importances=clf.feature_importances_
indices = np.argsort(importances)[::-1]

for f in range(X_train.shape[1]):
    print("%2d) %-*s %f" % (f + 1, 30,
    feat_labels[indices[f]],
    importances[indices[f]]))

 1) time_in_hospital_std           0.227079
 2) race_Caucasian                 0.094813
 3) Admission_Other                0.075396
 4) Discharge_Other                0.059969
 5) Reaction_None                  0.046318
 6) Medical_speciality_Missing     0.038662
 7) Pri_diag_Diabetes              0.036051
 8) Pri_diag_Others                0.035630
 9) Pri_diag_Digestive             0.033542
10) Medical_speciality_Other       0.033453
11) Reaction_Norm                  0.033241
12) Medical_speciality_InternalMedicine 0.033043
13) Medical_speciality_GeneralPractice 0.029450
14) Pri_diag_Respiratory           0.028861
15) race_Other                     0.027850
16) Pri_diag_Injury                0.022951
17) race_Missing                   0.021002
18) Pri_diag_Neoplasms             0.020337
19) Pri_diag_Genitourinary         0.019163
20) Reaction_High&Not              0.018878
21) Age_[60, 100)                  0.017092
22) Medical_speciality_Surgery     0.016564
23) Age_[30, 60)       

### Conclusiones:

* De la salida anteior se deduce que las variables con mayor poder discriminatorio a la hora de predecir si un paciente será reingresado son **time_in_hospital**, **race_Caucasian**, **dmission_Other** y **Discharge_Other**.


* Aunque el poder predictivo del árbol de decisión final es `aceptable`, este podría ser mejorado con un modelo de **redes neuronales recurrentes**, como lo proponen **Chopra et al.** (2017), pero con el inconveniente de ser menos interpretable en términos de la importancia de las variables.


* Chopra et al. (2017), realiza una comparación de varios métodos de clasificación; entre ellos Regresión logística, SVM, Árboles de decisión, Redes neuronales simples y Redes nuronales recurrentes, siendo este último método quien obtuvo el mejor comportamiento con un $AUC=0.8$ y un $ACC=81,12\%$ ........

### Referencias

* Chopra, C., Sinha, S., Jaroli, S., Shukla, A. and Maheshwari, S. (2017). **Recurrent Neural Networks with Non-Sequential Data to Predict Hospital Readmission of Diabetic Patients**. ICCBB 2017: Proceedings of the 2017 International Conference on Computational Biology and BioinformaticsOctober 2017 Pages 18–23.


* Strack, B., DeShazo, J. P., Gennings, C., Olmo, J. L., Ventura, S., Cios, K. J. and Clore, J. N. (2014). **Impact of hba1c measurement on hospital readmission rates: analysis of 70,000 clinical data base patient records**. BioMed research international. Volume 2014, Article ID 781670.


